In [40]:
pip install mysql-connector-python sqlalchemy pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pymysql

try:
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="Your_Password",
        database="project_analytics"
    )
    print("Connected successfully! 🎉")
except Exception as e:
    print("Connection failed:", e)


Connected successfully! 🎉


In [ ]:
from sqlalchemy import create_engine, text

# 1. Setup Connection
USERNAME = "root"              
PASSWORD = "Your_Password"        
HOST = "localhost"
DB = "project_analytics"

# Create connection string
connection_string = f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}/{DB}"
engine = create_engine(connection_string)

In [ ]:
import pandas as pd

# Loading with specific dtypes to handle the rating issues immediately
df_fragrance = pd.read_csv(
    "fragrance_combined_cleaned.csv",
    dtype={
        "Rating Value": "float64",
        "Rating Count": "float64"
    },
    low_memory=False
)

# Minimal cleaning
df_fragrance["Rating Count"] = df_fragrance["Rating Count"].fillna(0).astype(int)

# Rename columns (remove spaces)
df_fragrance.rename(columns={
    "Rating Value": "rating_value", 
    "Rating Count": "rating_count",
    "perfume_clean": "perfume_name",
    "brand_clean": "brand"
}, inplace=True)

print(f"Fragrance Data Ready: {df_fragrance.shape}")

# 3. Load Google Trends
df_trends = pd.read_csv("perfume_google_trends_clean.csv") # Ensure you use the CLEAN version filename
df_trends.rename(columns={"date": "trend_date", "keyword": "keyword_term"}, inplace=True)
print(f"Trends Data Ready: {df_trends.shape}")

# 4. Load Social Media
# You used 'aggregation_survey1_features.csv'
df_social = pd.read_csv("aggregated_survey1_features.csv") 
print(f"Social Data Ready: {df_social.shape}")

# 5. Push to MySQL (Using 'replace')
try:
    print("Uploading to MySQL...")
    
    df_fragrance.to_sql(name="perfumes", con=engine, if_exists="replace", index=False)
    print(" Table 'perfumes' created.")
    
    df_trends.to_sql(name="google_trends", con=engine, if_exists="replace", index=False)
    print(" Table 'google_trends' created.")
    
    df_social.to_sql(name="social_demographics", con=engine, if_exists="replace", index=False)
    print(" Table 'social_demographics' created.")
    
    print("\ SUCCESS: All datasets loaded into 'project_analytics'.")

except Exception as e:
    print(f"\n ERROR: {e}")

Fragrance Data Ready: (66871, 10)
Trends Data Ready: (85, 33)
Social Data Ready: (39, 42)
Uploading to MySQL...
✔ Table 'perfumes' created.
✔ Table 'google_trends' created.
✔ Table 'social_demographics' created.

🎉 SUCCESS: All datasets loaded into 'project_analytics'.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Setup Connection
USERNAME = "root"              
PASSWORD = "Your_Password"        
HOST = "localhost"
DB = "project_analytics"
engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}/{DB}")

# 2. Load the FEATURES file (contains 'mean_demand')
df_trend_features = pd.read_csv("perfume_trend_features_scaled.csv")

# 3. Rename columns. The file has 'keyword', 'mean_demand', etc.
df_trend_features.rename(columns={"keyword": "keyword_term"}, inplace=True)

# 4. Upload to MySQL as a NEW table called 'trend_features'
df_trend_features.to_sql(
    name="trend_features",
    con=engine,
    if_exists="replace",
    index=False
)

print(" Loaded 'trend_features' table (with mean_demand) -> MySQL ✔")

🎉 Loaded 'trend_features' table (with mean_demand) -> MySQL ✔


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Setup Connection
USERNAME = "root"              
PASSWORD = "Your_Password"        
HOST = "localhost"
DB = "project_analytics"
engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}/{DB}")

# 2. Load the Features File
df_trend_features = pd.read_csv("perfume_trend_features_scaled.csv")

# 3. FIX: Rename 'Unnamed: 0' to 'keyword_term'
df_trend_features.rename(columns={
    "Unnamed: 0": "keyword_term", 
    "keyword": "keyword_term"
}, inplace=True)

# 4. Upload to MySQL (Replace existing)
df_trend_features.to_sql(
    name="trend_features",
    con=engine,
    if_exists="replace",
    index=False
)

print(" Fixed 'trend_features' table! Column 'keyword_term' should now exist.")

🎉 Fixed 'trend_features' table! Column 'keyword_term' should now exist.
